In [1]:
import cv2
from pathlib import Path
from tqdm import tqdm
import os

In [2]:
out_dir = 'custom_train/datasets/OurDataSet/'
clips_dir = 'inputs/media/Clips/'
annotations_dir = 'inputs/annotations/'

classes = ['car', 'motorcycle', 'truck', 'priority', 'bus']

rel_dir = 'datasets/OurDataSet/'

images_out_dir = out_dir + 'images/'
labels_out_dir = out_dir + 'labels/'


In [3]:
yam_dir = out_dir + 'OurDataset.yaml'
train_dir = rel_dir + 'images/'+'train/'
val_dir = rel_dir + 'images/' + 'val/'

nc = len(classes)

In [4]:
videos = list(Path(clips_dir).rglob('*.mp4'))
videos = [str(v) for v in videos]
# videos = [str(v) for v in videos if 'train' in str(v).lower()]


In [5]:
train_paths = []
val_paths = []

for video in videos:
    # print(video)
    extension = ''
    if 'train' in video.lower():
        extension = '_Train'
    elif 'test' in video.lower():
        extension = '_Test'
    elif 'valid' in video.lower():
        extension = '_Val'

    vid_dir = video.replace('.mp4', extension)

    vid_dir = vid_dir.split('\\')[-1].split('/')[-1]
    annotations_paths = [str(v) for v in list(Path(annotations_dir+vid_dir).rglob('*.txt'))]

    cap = cv2.VideoCapture(video)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps)
    # print(n_frames)
    for i in tqdm(range(n_frames), desc='Processing '+vid_dir):
        ret, frame = cap.read()
        if ret:
            # creaye dir if not exist
            if not Path(images_out_dir+vid_dir).exists():
                Path(images_out_dir+vid_dir).mkdir(parents=True)
            
            # ensure frame is no zero size array
            if frame.shape[0] == 0 or frame.shape[1] == 0 or frame.size == 0:
                print('SKIPPING ERRORED FRAME: ', vid_dir, i)
                continue
            
            contents = open(annotations_paths[i], 'r').readlines()
            if len(contents) == 0:
                continue
            
            if not Path(labels_out_dir + vid_dir).exists():
                Path(labels_out_dir + vid_dir).mkdir(parents=True)

            dest_path = labels_out_dir + vid_dir+'/' + str(i) + '.txt'
            with open(dest_path, 'w') as f:
                f.writelines(contents)

            cv2.imwrite(images_out_dir + vid_dir+'/' + str(i) + '.jpg', frame)

        else:
            print('error')
            break

Processing Wet_Dim_Val: 100%|██████████| 317/317 [00:05<00:00, 54.70it/s]


In [11]:
train_paths = []
val_paths = []

full_images = out_dir + 'images/'
full_labels = out_dir + 'labels/'

all_image_dirs = os.listdir(full_images)
all_train_images = [i for i in all_image_dirs if 'train' in i.lower()]
all_test_images = [i for i in all_image_dirs if 'test' in i.lower()]
all_val_images = [i for i in all_image_dirs if 'val' in i.lower()]

all_train_images

path = '../datasets/OurDataSet/'
im_dir ='images/'
lab_dir = 'labels/'


In [14]:
yam_file = open(yam_dir, 'w')
yam_file.write('# This file is automatically generated by dataSetter.py\n')
# yam_file.write('train: ' + train_dir + '\n')
# yam_file.write('val: ' + val_dir + '\n')

yam_file.write('path: ' + path + '\n')
yam_file.write('train: ' + '\n')
for dir in all_train_images:
    yam_file.write('  - '+im_dir+dir+'\n')
if len(all_val_images) > 0:
    yam_file.write('val: ' + '\n')
    for dir in all_val_images:
        yam_file.write('  - '+im_dir+dir+'\n')
yam_file.write('\n')
if len(all_test_images) > 0:
    yam_file.write('test: ' + '\n')
    for dir in all_test_images:
        yam_file.write('  - '+im_dir+dir+'\n')

yam_file.write('\n')


yam_file.write('# number of classes\n')
yam_file.write('nc: ' + str(nc) + '\n')
yam_file.write('\n')
yam_file.write('# class names\n')
yam_file.write('names: ')
yam_file.write('[' + '\'' + '\', \''.join(classes) + '\'' + ']')


49

# Testing

In [ ]:
out_ds_path = 'custom_train/datasets/OurDataSet/'
images = [str(s) for s in Path(out_ds_path).rglob('**/*.jpg')]

classes = ['car', 'motorbike', 'truck', 'priority', 'bus']


In [ ]:


for img_ix, img in enumerate(images):
    original_image = cv2.imread(img)

    label_dir = img.replace('.jpg', '.txt').replace('images', 'labels')

    if os.path.exists(label_dir):
        coords_all = open(label_dir, 'r').read().split('\n')
        for coords in coords_all:
            coords = coords.split(' ')
            class_id = coords[0]
            class_name = classes[int(class_id)]

            coords = [float(x) for x in coords[1:]]

            image_x = original_image.shape[1]
            image_y = original_image.shape[0]
            coords[0] = coords[0] * image_x
            coords[1] = coords[1] * image_y
            coords[2] = coords[2] * image_x
            coords[3] = coords[3] * image_y

            # coords = x_mid, y_mid, w, h, convert them to x_min, y_min, x_max, y_max
            coords = [coords[0] - coords[2]/2, coords[1] - coords[3] /
                      2, coords[0] + coords[2]/2, coords[1] + coords[3]/2]

            x, y, w, h = coords

            cv2.rectangle(original_image, (int(coords[0]), int(
                coords[1])), (int(coords[2]), int(coords[3])), (0, 255, 0), 2)
            cv2.putText(original_image, class_name, (int(coords[0]), int(
                coords[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('image', original_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print('Label dir: ', label_dir)
            print('Image: ', img)
            print('CoordsAll: ', coords_all)
            print('Coords: ', x, y, w, h)
            print('Image size: ', original_image.shape)

            break
        continue

    print('Showing no labels')
    cv2.imshow('image', original_image)
    cv2.waitKey(0)
    continue


cv2.destroyAllWindows()
